<a href="https://colab.research.google.com/github/marcpadro/eines_colab/blob/main/YOLO9000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Inicialitzar
# make sure you use Tesla K80 GPU (12GB), by changing 'runtmie type'

!nvidia-smi

!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

!apt install ffmpeg libopencv-dev libgtk-3-dev python-numpy python3-numpy libdc1394-22 libdc1394-22-dev libjpeg-dev libtiff5-dev libavcodec-dev libavformat-dev libswscale-dev libxine2-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libv4l-dev libtbb-dev qtbase5-dev libfaac-dev libmp3lame-dev libopencore-amrnb-dev libopencore-amrwb-dev libtheora-dev libvorbis-dev libxvidcore-dev x264 v4l-utils unzip

#clone darknet repository
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
%cd -q /content
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet.git

# go to the darknet folder, enable GPU, then compile the entire thing
%cd -q darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

# get yolo9000 weights
!wget https://pjreddie.com/media/files/yolo9000.weights
!wget -O ./data/9k.labels https://raw.githubusercontent.com/pjreddie/darknet/master/data/9k.labels
!wget -O ./data/9k.names https://raw.githubusercontent.com/pjreddie/darknet/master/data/9k.names
!chmod a+x ./darknet

from google.colab import files
from google.colab import output

output.clear()

print("Ready!")



In [ ]:
#@title Processar vídeo(s)

!rm -fr /content/input_files
!rm -fr /content/output_files

!mkdir /content/input_files
!mkdir /content/output_files

%cd -q /content/input_files
uploaded = files.upload()

%cd -q /content/darknet

thresh = 0.2 #@param {type: 'slider', min: 0, max: 1, step: 0.05}

# show_confidence = False #@param {type: 'boolean'}

for file in uploaded:
  input_file = "/content/input_files/{}".format(file)
  output_file = "/content/output_files/yolo9000__t{}_{}".format(thresh, file)
  !./darknet detector demo cfg/combine9k.data cfg/yolo9000.cfg yolo9000.weights -dont_show {input_file} -i 0 -out_filename {output_file} -thresh {thresh}
  files.download(output_file)

output.clear()

print("Done!")

In [ ]:
#@title Processar imatge(s)

!rm -fr /content/input_files
!rm -fr /content/output_files

!mkdir /content/input_files
!mkdir /content/output_files

%cd -q /content/input_files
uploaded = files.upload()

%cd -q /content/darknet

thresh = 0.2 #@param {type: 'slider', min: 0, max: 1, step: 0.05}

for file in uploaded:
  input_file = "/content/input_files/{}".format(file)
  output_file = "/content/output_files/yolo9000__t{}_{}".format(thresh, file)
  !./darknet detector test cfg/combine9k.data cfg/yolo9000.cfg yolo9000.weights -dont_show {input_file} -thresh {thresh}
  !mv ./predictions.jpg {output_file}
  files.download(output_file)

output.clear()

print("Done!")
